## Import

In [1]:
# preprocessing
import numpy as np
import pandas as pd
import tqdm
import random
import os

# imputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

# model learning
from sklearn.model_selection import train_test_split
from supervised.automl import AutoML
from sklearn.utils.class_weight import compute_sample_weight
import statsmodels.api as sm

# 평가 지표
from sklearn.metrics import mean_absolute_error

# 모델 저장
import pickle

## Fixed Random-Seed

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

## Load Data

In [3]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')
building_info = pd.read_csv('../data/building_info.csv')

## Change Names

In [4]:
translation_dict = {
    '건물기타': 'Other Buildings',
    '공공': 'Public',
    '대학교': 'University',
    '데이터센터': 'Data Center',
    '백화점및아울렛': 'Department Store and Outlet',
    '병원': 'Hospital',
    '상용': 'Commercial',
    '아파트': 'Apartment',
    '연구소': 'Research Institute',
    '지식산업센터': 'Knowledge Industry Center',
    '할인마트': 'Discount Mart',
    '호텔및리조트': 'Hotel and Resort'
}

building_info['건물유형'] = building_info['건물유형'].replace(translation_dict)
# building_info.drop('Unnamed: 0', axis = 1 , inplace=True)

In [5]:
train_df.columns = ['num_date_time', '건물번호', '일시', '기온', '강수량', '풍속', '습도',
       '일조', '일사', '전력소비량']

In [6]:
test_df.columns = ['num_date_time', '건물번호', '일시', '기온', '강수량', '풍속', '습도']

In [7]:
building_info.columns = ['건물번호', '건물유형', '연면적', '냉방면적', '태양광용량', 'ESS저장용량',
       'PCS용량']

## Encoding

In [8]:
# one hot encoding 생성
building_info = pd.get_dummies(building_info, columns=['건물유형'], drop_first=True)

In [9]:
# LabelEncoder를 객체로 생성
#encoder = LabelEncoder()

# fit, transform 메소드를 통한 레이블 인코딩
#encoder.fit(building_info['건물유형'])
#building_info['건물유형'] = encoder.transform(building_info['건물유형'])

## Merge building info

In [10]:
building_info['태양광용량'][building_info['태양광용량'] == '-'] = np.nan
building_info['ESS저장용량'][building_info['ESS저장용량'] == '-'] = np.nan
building_info['PCS용량'][building_info['PCS용량'] == '-'] = np.nan

In [11]:
train_df = pd.merge(train_df, building_info, on='건물번호', how='left')
test_df = pd.merge(test_df, building_info, on='건물번호', how='left')

## Train Data Pre-Processing

In [12]:
train_df.isna().sum()

num_date_time                            0
건물번호                                     0
일시                                       0
기온                                       0
강수량                                 160069
풍속                                      19
습도                                       9
일조                                   75182
일사                                   87913
전력소비량                                    0
연면적                                      0
냉방면적                                     0
태양광용량                               130560
ESS저장용량                             193800
PCS용량                               193800
건물유형_Commercial                          0
건물유형_Data Center                         0
건물유형_Department Store and Outlet         0
건물유형_Discount Mart                       0
건물유형_Hospital                            0
건물유형_Hotel and Resort                    0
건물유형_Knowledge Industry Center           0
건물유형_Other Buildings                     0
건물유형_Public

- 결측치가 시급하다. 아무리봐도 처리 해야할 것 같다.

In [13]:
#시계열 특성을 학습에 반영하기 위해 일시를 월, 일, 시간으로 나눕니다
train_df['월'] = train_df['일시'].apply(lambda x : int(x[4:6]))
#train_df['일'] = train_df['일시'].apply(lambda x : int(x[6:8]))
train_df['시'] = train_df['일시'].apply(lambda x : int(x[9:11]))

In [14]:
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')

In [15]:
# 요일 추가 (0이 월요일 6이 일요일)
train_df['주'] = train_df['일시'].dt.dayofweek

In [16]:
# 변수 정리
train_df2 = train_df.drop(columns=['num_date_time', '일시', '일조', '일사',
                                 '태양광용량','ESS저장용량','PCS용량', '강수량'])

In [17]:
# 강수량 결측치는 0으로 보정 -> 비 안왔을것이라 예상하고 진행함
# train_df2['강수량(mm)'][train_df2['강수량(mm)'].isna()] = 0

In [18]:
# imputation for mice
#imputer_mice = IterativeImputer(random_state=42)
#imputer_mice.fit(train_df2)

# 데이터 변환 (array로 반환하기 때문에 필요에 맞는 형태로 변환 후 사용)
#train_df2 = pd.DataFrame(imputer_mice.transform(train_df2),
#                         columns=train_df2.columns)

In [19]:
# 그냥 0으로 채우기
train_df2[train_df2.isna()] = 0

## Append Value

In [20]:
# 온도에 따른 포화 수증기압(mb)
#train_df2['포화수증기압'] = 6.11*10**((7.5*train_df2['기온'])/(237.3 + train_df2['기온']))

In [21]:
# 현재 대기의 수증기압(mb)
#train_df2['대기압'] = train_df2['습도'] * train_df2['포화수증기압'] / 100

In [22]:
# 절대습도(g/m^3)
#train_df2['절대습도'] = (0.794*train_df2['대기압'])/(1+0.00366*train_df2['기온'])

In [23]:
# 공기중의 혼합비(mb)
#train_df2['혼합비'] = 0.622 * train_df2['대기압']/(1013.25 - train_df2['대기압'])

In [24]:
# 습구온도
#train_df2['습구온도'] = train_df2['기온']*np.arctan(0.151977+(train_df2['습도'] + 8.313659)**0.5) + np.arctan(train_df2['기온'] + train_df2['습도']) - np.arctan(train_df2['습도'] - 1.676331) +0.00391838*(train_df2['습도']**1.5)*np.arctan(0.023101*train_df2['습도'])-4.686035

In [25]:
# 불쾌지수
#train_df2['불쾌지수'] = 9/5*train_df2['기온'] - 0.55*(1-0.01*train_df2['습도'])*(9/5*train_df2['기온'] - 26) + 32

In [26]:
# 더위체감지수
#train_df2['더위체감지수'] = -0.24418 + 0.553991*train_df2['습구온도'] + 0.455346*train_df2['기온'] - 0.00217*train_df2['습구온도']**2 + 0.002782*train_df2['습구온도']*train_df2['기온'] + 3

In [27]:
# np.select
#condlist = [
#    (train_df2['더위체감지수'] < 21),
#    (train_df2['더위체감지수'] < 25) & (train_df2['더위체감지수'] >= 21),
#    (train_df2['더위체감지수'] < 28) & (train_df2['더위체감지수'] >= 25),
#    (train_df2['더위체감지수'] < 31) & (train_df2['더위체감지수'] >= 28),
#    (train_df2['더위체감지수'] >= 31)
#            ]

#choicelist = [0, 1, 2, 3, 4]

In [28]:
# 범주형 체감지수
#train_df2['더위체감지수_범주'] = np.select(condlist, choicelist)

In [29]:
# 열지수
#train_df2['열지수'] = (5/9)*(-42.379 + (2.04901523 * ((9/5)*train_df2['기온']+32)) + (10.14333127 * train_df2['습도']) - (0.22475541 * ((9/5)*train_df2['기온']+32)*train_df2['습도']) - (6.83783e-3 * ((9/5)*train_df2['기온']+32)**2) - (5.481717e-2 * train_df2['습도']**2) + (1.22874e-3 * ((9/5)*train_df2['기온']+32)**2*train_df2['습도']) + (8.5282e-4 * ((9/5)*train_df2['기온']+32)*train_df2['습도']**2) - (1.99e-6 * ((9/5)*train_df2['기온']+32)**2*train_df2['습도']**2) - 32)

In [30]:
# 평균냉방면적
train_df2['평균냉방면적'] = train_df2['냉방면적']/train_df2['연면적']

In [31]:
# onehot 주
train_df2 = pd.get_dummies(train_df2, columns=['주'], drop_first=True)

In [32]:
# 훈련 및 테스트 데이터 셋
train_x = train_df2[train_df2.columns.difference(['전력소비량'])]
train_y = train_df2['전력소비량']

# BoxCox

In [33]:
# 가중치
boxcox = 0.21

In [34]:
# 적용
train_y_root_twice = train_y**boxcox

## Value Check

In [35]:
results = sm.OLS(train_y_root_twice, sm.add_constant(train_x)).fit()

In [36]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  전력소비량   R-squared:                       0.386
Model:                            OLS   Adj. R-squared:                  0.386
Method:                 Least Squares   F-statistic:                     4939.
Date:                Sun, 06 Aug 2023   Prob (F-statistic):               0.00
Time:                        18:53:09   Log-Likelihood:            -1.9306e+05
No. Observations:              204000   AIC:                         3.862e+05
Df Residuals:                  203973   BIC:                         3.864e+05
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
const                                5.6244      0.038    146.745      0.000       5.549       5.700
건물번호                                -0.0452      0.001    -87.023      0.000      -0.046      -0.044
건물유형_Commercial                      0.3386      0.008     40.395      0.000       0.322       0.355
건물유형_Data Center                     0.2338      0.018     13.112      0.000       0.199       0.269
건물유형_Department Store and Outlet    -0.5353      0.014    -37.256      0.000      -0.563      -0.507
건물유형_Discount Mart                   1.3742      0.014     95.278      0.000       1.346       1.402
건물유형_Hospital                        0.2322      0.011     21.004      0.000       0.211       0.254
건물유형_Hotel and Resort                1.7242      0.018     95.324      0.000       1.689       1.760
건물유형_Knowledge Industry Center       1.2199      0.011    112.646      0.000       1.199       1.241
건물유형_Other Buildings                -1.9755      0.030    -65.772      0.000      -2.034      -1.917
건물유형_Public                         -1.6460      0.025    -67.116      0.000      -1.694      -1.598
건물유형_Research Institute              1.2458      0.008    150.080      0.000       1.230       1.262
건물유형_University                     -0.4542      0.020    -22.237      0.000      -0.494      -0.414
기온                                   0.0310      0.001     59.469      0.000       0.030       0.032
냉방면적                             -2.418e-06   4.39e-08    -55.071      0.000    -2.5e-06   -2.33e-06
습도                                  -0.0027      0.000    -23.044      0.000      -0.003      -0.002
시                                    0.0116      0.000     54.225      0.000       0.011       0.012
연면적                               2.072e-06   3.73e-08     55.597      0.000       2e-06    2.14e-06
월                                    0.0406      0.002     17.840      0.000       0.036       0.045
주_1                                  0.0442      0.005      8.492      0.000       0.034       0.054
주_2                                  0.0625      0.005     12.191      0.000       0.052       0.073
주_3                                  0.0858      0.005     16.447      0.000       0.076       0.096
주_4                                  0.0684      0.005     13.142      0.000       0.058       0.079
주_5                                 -0.0609      0.005    -11.651      0.000      -0.071      -0.051
주_6                                 -0.1062      0.005    -20.383      0.000      -0.116      -0.096
평균냉방면적                               0.5346      0.009     60.841      0.000       0.517       0.552
풍속                                   0.0679      0.001     61.785      0.000       0.066       0.070
=====================================================

## Standard Scaling

In [37]:
# 표준화
#std_scaler = StandardScaler()
#train_x = std_scaler.fit_transform(train_x)

## Regression Model Fit

In [38]:
# automl modeling
automl = AutoML(mode="Compete",
                total_time_limit = 3600*3,
                algorithms=["Baseline",
                            "CatBoost",
                            "Xgboost",
                            "Random Forest",
                            "Extra Trees",
                            "LightGBM",
                            "Neural Network"], 
                ml_task = "regression",
                eval_metric = 'mae',
                random_state = 42,
                results_path = '../model/AutoML_OneHot_0.21_NA0_1value_week/'
               )

In [39]:
# fitting
automl.fit(train_x, train_y_root_twice)

AutoML directory: ../model/AutoML_OneHot_0.21_NA0_1value_week/
The task is regression with evaluation metric mae
AutoML will use algorithms: ['Baseline', 'CatBoost', 'Xgboost', 'Random Forest', 'Extra Trees', 'LightGBM', 'Neural Network']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree mae 0.431553 trained in 0.89 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
* Step simple_algorithms will try to check up to 1 model
1_Baseline mae 0.620316 trained in 3.64 seconds
* Step default_algorithms will try to check up to 6 models
2_Default_LightGBM mae 0.046592 trained in 920.16 seconds
3_Defa

AutoML(algorithms=['Baseline', 'CatBoost', 'Xgboost', 'Random Forest',
                   'Extra Trees', 'LightGBM', 'Neural Network'],
       eval_metric='mae', ml_task='regression', mode='Compete', random_state=42,
       results_path='../model/AutoML_OneHot_0.21_NA0_1value_week/',
       total_time_limit=10800)

## Test Data Pre-Processing

In [40]:
test_df['월'] = test_df['일시'].apply(lambda x : int(x[4:6]))
#test_df['일'] = test_df['일시'].apply(lambda x : int(x[6:8]))
test_df['시'] = test_df['일시'].apply(lambda x : int(x[9:11]))

In [41]:
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

In [42]:
# 요일 추가 (0이 월요일 6이 일요일)
test_df['주'] = test_df['일시'].dt.dayofweek

In [43]:
# 변수 정리
test_df2 = test_df.drop(columns=['num_date_time', '일시',
                                 '태양광용량','ESS저장용량','PCS용량','강수량'])

- test는 결측치가 없기 때문에 impute 안해도됨.

## Append Value

In [44]:
# 온도에 따른 포화 수증기압(mb)
#test_df2['포화수증기압'] = 6.11*10**((7.5*test_df2['기온'])/(237.3 + test_df2['기온']))

In [45]:
# 현재 대기의 수증기압(mb)
#test_df2['대기압'] = test_df2['습도'] * test_df2['포화수증기압'] / 100

In [46]:
# 절대습도(g/m^3)
#test_df2['절대습도'] = (0.794*test_df2['대기압'])/(1+0.00366*test_df2['기온'])

In [47]:
# 공기중의 혼합비(mb)
# test_df2['혼합비'] = 0.622 * test_df2['대기압']/(1013.25 - test_df2['대기압'])

In [48]:
# 습구온도
#test_df2['습구온도'] = test_df2['기온']*np.arctan(0.151977+(test_df2['습도'] + 8.313659)**0.5) + np.arctan(test_df2['기온'] + test_df2['습도']) - np.arctan(test_df2['습도'] - 1.676331) +0.00391838*(test_df2['습도']**1.5)*np.arctan(0.023101*test_df2['습도'])-4.686035

In [49]:
# 불쾌지수
#test_df2['불쾌지수'] = 9/5*test_df2['기온'] - 0.55*(1-0.01*test_df2['습도'])*(9/5*test_df2['기온'] - 26) + 32

In [50]:
# 더위체감지수
#test_df2['더위체감지수'] = -0.24418 + 0.553991*test_df2['습구온도'] + 0.455346*test_df2['기온'] - 0.00217*test_df2['습구온도']**2 + 0.002782*test_df2['습구온도']*test_df2['기온'] + 3

In [51]:
# np.select
#condlist = [
#    (test_df2['더위체감지수'] < 21),
#    (test_df2['더위체감지수'] < 25) & (test_df2['더위체감지수'] >= 21),
#    (test_df2['더위체감지수'] < 28) & (test_df2['더위체감지수'] >= 25),
#    (test_df2['더위체감지수'] < 31) & (test_df2['더위체감지수'] >= 28),
#    (test_df2['더위체감지수'] >= 31)
#           ]

#choicelist = [0, 1, 2, 3, 4]

In [52]:
# 범주형 체감지수
#test_df2['더위체감지수_범주'] = np.select(condlist, choicelist)

In [53]:
# 열지수
#test_df2['열지수'] = (5/9)*(-42.379 + (2.04901523 * ((9/5)*test_df2['기온']+32)) + (10.14333127 * test_df2['습도']) - (0.22475541 * ((9/5)*test_df2['기온']+32)*test_df2['습도']) - (6.83783e-3 * ((9/5)*test_df2['기온']+32)**2) - (5.481717e-2 * test_df2['습도']**2) + (1.22874e-3 * ((9/5)*test_df2['기온']+32)**2*test_df2['습도']) + (8.5282e-4 * ((9/5)*test_df2['기온']+32)*test_df2['습도']**2) - (1.99e-6 * ((9/5)*test_df2['기온']+32)**2*test_df2['습도']**2) - 32)

In [54]:
# 평균냉방면적
test_df2['평균냉방면적'] = test_df2['냉방면적']/test_df2['연면적']

In [55]:
# onehot 주
test_df2 = pd.get_dummies(test_df2, columns=['주'], drop_first=True)

In [56]:
# test_df2 = std_scaler.fit_transform(test_df2)

In [57]:
# 훈련 및 테스트 데이터 셋
#test_df2 = test_df2[test_df2.columns.difference(['습구온도', '더위체감지수'])]

## Inference

In [58]:
preds = automl.predict(test_df2)

## Submission

In [59]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,num_date_time,answer
0,1_20220825 00,0
1,1_20220825 01,0
2,1_20220825 02,0
3,1_20220825 03,0
4,1_20220825 04,0
...,...,...
16795,100_20220831 19,0
16796,100_20220831 20,0
16797,100_20220831 21,0
16798,100_20220831 22,0


In [60]:
submission['answer'] = preds**(1/boxcox)
submission

,num_date_time,answer
0,1_20220825 00,1998.998464
1,1_20220825 01,1893.161284
2,1_20220825 02,1738.021863
3,1_20220825 03,1661.430372
4,1_20220825 04,1684.264028
...,...,...
16795,100_20220831 19,901.216848
16796,100_20220831 20,811.503202
16797,100_20220831 21,705.885176
16798,100_20220831 22,601.231108


In [61]:
submission.to_csv('../data/AutoML_OneHot_0.21_NA0_1value_week.csv', index=False)